In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [ ]:
from multiversx_sdk_core import Address
from multiversx_sdk_core.transaction_builders import (ContractCallBuilder, ESDTNFTTransferBuilder, 
                                                      ContractDeploymentBuilder, ContractUpgradeBuilder)
from multiversx_sdk_core.transaction_builders import DefaultTransactionBuildersConfiguration
from multiversx_sdk_core import CodeMetadata
from multiversx_sdk_core import TokenPayment
from multiversx_sdk_network_providers import ProxyNetworkProvider
from tools.notebooks import env
from multiversx_sdk_core import AccountNonceHolder
from multiversx_sdk_wallet import UserSigner

config = DefaultTransactionBuildersConfiguration(chain_id=env.CHAIN_ID)
proxy = ProxyNetworkProvider(env.PROXY_URL)

In [ ]:

contract_address = env.FORWARDER_CONTRACT


deploy

In [ ]:
user_signer = UserSigner.from_pem_file(Path(env.ADMIN_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(env.ADMIN).nonce)
metadata = CodeMetadata(upgradeable=True, readable=True, payable=False, payable_by_contract=True)
path = Path.home() / "projects" / "contracts" / "mx-sc-dummy-proxy" / "dummy-proxy" / "output" / "dummy-proxy.wasm"

builder = ContractDeploymentBuilder(
        config=config,
        owner=env.ADMIN,
        gas_limit=200000000,
        code_metadata=metadata,
        code=path.read_bytes(),
        deploy_arguments=[]
    )

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

Upgrade

In [ ]:
user_signer = UserSigner.from_pem_file(Path(env.ADMIN_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(env.ADMIN).nonce)
metadata = CodeMetadata(upgradeable=True, readable=True, payable=False, payable_by_contract=True)
path = Path.home() / "projects" / "dex" / "dex-v2" / "sc-dex-rs"/ "dex"/ "dummy-proxy" / "output" / "dummy-proxy.wasm"

builder = ContractUpgradeBuilder(
        config=config,
        owner=env.ADMIN,
        contract=contract_address,
        gas_limit=200000000,
        code_metadata=metadata,
        code=path.read_bytes(),
        upgrade_arguments=[]
    )

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

### endpoint_call

migrate lkmex

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(env.LKMEX, 4580728, 1000000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callTransferEndpoint",
    caller=user,
    call_arguments=[env.ENERGY_CONTRACT, "migrateOldTokens"],
    gas_limit=50000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

lock for more direct send

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(env.XMEX, 46, 1000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callTransferEndpoint",
    caller=user,
    call_arguments=[env.ENERGY_CONTRACT, "lockTokens", 1440],
    gas_limit=50000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

lock for more internal send

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callInternalTransferEndpoint",
    caller=user,
    call_arguments=[env.XMEX, int("0x44", base=16), 100000000000000000000, 
                    env.ENERGY_CONTRACT, "lockTokens", 1440],
    gas_limit=50000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

reduce lock direct send

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(env.XMEX, 46, 1000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callTransferEndpoint",
    caller=user,
    call_arguments=[env.ENERGY_CONTRACT, "reduceLockPeriod", 720],
    gas_limit=50000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

farm stuff

In [ ]:
user = env.USER3
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_integer("LKFARM-9d1ea8", int("904cf3", 16), 1000000000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callTransferEndpoint",
    caller=user,
    call_arguments=[Address.from_bech32("erd1qqqqqqqqqqqqqpgqrc4pg2xarca9z34njcxeur622qmfjp8w2jps89fxnl"), 
                    "exitFarmProxy", 
                    Address.from_bech32("erd1qqqqqqqqqqqqqpgqyawg3d9r4l27zue7e9sz7djf7p9aj3sz2jpsm070jf")],
    gas_limit=50000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

In [ ]:
user = env.USER2
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
pair_contract = Address.from_bech32("erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2")

transfers = [
    TokenPayment.fungible_from_amount(env.WEGLD, 200000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=pair_contract,
    function_name="swapTokensFixedInput",
    caller=user,
    call_arguments=[env.MEX, 1],
    esdt_transfers=transfers,
    gas_limit=50000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

liquidity stuff

In [ ]:
user = env.USER3
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callInternalTransferEndpoint",
    caller=user,
    call_arguments=["LKLP-03a2fa", int("0f53f7", base=16), 10000, 
                    Address.from_bech32("erd1qqqqqqqqqqqqqpgqrc4pg2xarca9z34njcxeur622qmfjp8w2jps89fxnl"), 
                    "removeLiquidityProxy", 
                    Address.from_bech32("erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2"),
                    1, 1],
    gas_limit=50000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

reduce lock internal send

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callInternalTransferEndpoint",
    caller=user,
    call_arguments=[env.XMEX, int("0x5b", base=16), 1000000000000000000000, 
                    env.ENERGY_CONTRACT, "reduceLockPeriod", 720],
    gas_limit=50000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

lock free mex

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.fungible_from_amount(env.MEX, 1000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callTransferEndpoint",
    caller=user,
    call_arguments=[env.ENERGY_CONTRACT, "lockTokens", 1440],
    gas_limit=50000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

claim farm rewards

In [ ]:
user = env.USER3
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_integer("EGLDUSDCFL-45109b", int("4367", 16), 1000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=env.FORWARDER_CONTRACT,
    function_name="callTransferEndpoint",
    caller=user,
    call_arguments=[Address.from_bech32("erd1qqqqqqqqqqqqqpgqv0pz5z3fkz54nml6pkzzdruuf020gqzykp2sya7kkv"),
                    "claimRewards"],
    gas_limit=50000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

escrow: transfer tokens

contract to account

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(env.XMEX, 46, 1000, 18),
    TokenPayment.meta_esdt_from_amount(env.XMEX, 47, 1000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callTransferEndpoint",
    caller=user,
    call_arguments=[env.ESCROW_CONTRACT, "lockFunds", env.USER2],
    gas_limit=50000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

account to contract

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(env.XMEX, 46, 2000, 18),
    TokenPayment.meta_esdt_from_amount(env.XMEX, 47, 2000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=env.ESCROW_CONTRACT,
    function_name="lockFunds",
    caller=user,
    call_arguments=[contract_address],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

withdraw

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callEndpoint",
    caller=user,
    call_arguments=[env.ESCROW_CONTRACT, "withdraw", user],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

wrapper contract

wrap direct

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(env.XMEX, 46, 1000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callTransferEndpoint",
    caller=user,
    call_arguments=[env.WRAPPER_CONTRACT, "wrapLockedToken"],
    gas_limit=50000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

wrap internal send

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callInternalTransferEndpoint",
    caller=user,
    call_arguments=[env.XMEX, int("0x5b", base=16), 200000000000000000000, 
                    env.WRAPPER_CONTRACT, "wrapLockedToken"],
    gas_limit=50000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

unwrap

In [ ]:
user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(env.WXMEX, 1, 1000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="callTransferEndpoint",
    caller=user,
    call_arguments=[env.WRAPPER_CONTRACT, "unwrapLockedToken"],
    gas_limit=50000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)